In [1]:
import numpy as np 
import pandas as pd 

In [93]:
import tensorflow as tf
import keras 
import os
import PIL      #inplace of OpenCV
import matplotlib.pyplot as plt
#import sklearn

In [3]:
os.getcwd()        #loads current working directory

In [4]:
path='../input/siim-isic-melanoma-classification/jpeg/train'

In [5]:
len(os.listdir(path))

In [6]:
df=pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
df

In [7]:
df['benign_malignant'].unique()

In [8]:
pd.value_counts(df['benign_malignant'])

In [9]:
from PIL import Image

In [10]:
img2=Image.open('../input/siim-isic-melanoma-classification/jpeg/train/ISIC_0052212.jpg')
img2

In [11]:
img2=np.asarray(img2)           # in order to plot we need to convert in to an array form
plt.imshow(img2)

In [12]:
image=os.listdir(path)
plt.figure(figsize=(10,6))
for i in range(15):
    img=Image.open(path+"/"+image[i])
    img=np.asarray(img)
    plt.subplot(3,5,i+1)
    plt.title(df['benign_malignant'][i])
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img)
plt.show()
    
    
    

In [87]:
train_datagen=keras.preprocessing.image.ImageDataGenerator(       #parameters copy pasted from https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
    rescale=1/255.0,
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.0,

)

test_datagen=keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255
)


In [89]:
train_path='../input/siim-isic-melanoma-classification/jpeg/train'
train_image=os.listdir(train_path)
train_image.sort()               #because data was unsorted 


In [90]:
df.sort_values('image_name',inplace=True)

In [91]:
data={"imgpath":[],"output":[]}
for i in range(len(train_image)):
    data['imgpath'].append(path+'/'+image[i])
    data['output'].append(df['benign_malignant'][i])
    
data_df=pd.DataFrame(data)
data_df

In [92]:
df1=pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
df1

In [23]:
# test datset doesn't contain Target so we would have to use Training images and divide it into training and testing dataset

In [94]:
from sklearn.model_selection import train_test_split

In [83]:
x_train, x_test, y_train, y_test=train_test_split(data_df['imgpath'],data_df['output'],test_size=0.25)

In [84]:
train_df=pd.DataFrame(x_train)
train_df['output']=y_train
train_df

In [85]:
test_df=pd.DataFrame(x_test)
test_df['output']=y_test
test_df

In [97]:
train_gen = train_datagen.flow_from_dataframe(train_df, 
                                              x_col='imgpath',
                                              y_col='output',
                                              target_size=(299,299),
                                              batch_size=256,
                                              class_mode='binary',
                                             )

test_gen = test_datagen.flow_from_dataframe(test_df,
                                            x_col='imgpath',
                                            y_col='output',
                                            target_size=(299,299),
                                            shuffle=False,
                                            batch_size=256,
                                            class_mode='binary',
                                           )

In [98]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [99]:
model = Sequential([
    Conv2D(32, 3, input_shape=(299,299,3), activation='relu'),
    MaxPooling2D(),
    Conv2D(16, 3, activation='relu'),
    MaxPooling2D(),
    Conv2D(16, 3, activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [100]:
model.summary()

In [102]:
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])

In [ ]:
model_fit = model.fit(train_gen, epochs=10, validation_data=test_gen)